Implementation of Logistic Regression

In [4]:
# open the file
import pandas as pd
with open("testSet (2).txt","r") as file:
  emails = [i.split("\n") for i in file]


positive = emails[0:2989]
negative = emails[2989:]

pos = [1]*2989
neg = [0]*1332

labels = pos + neg

#prepare the data

data = pd.DataFrame(zip(emails,labels), columns = ["emails","labels"])
data.head()

,emails,labels
0,[Fantastic spot for an even or a quite cocktai...,1
1,"[Love, love, love the calamari. It's so good a...",1
2,[Love this place. Stiff martinis and cocktails...,1
3,[It's everything a great cocktail bar should b...,1
4,"[I came here before a pirates game, so it was ...",1


In [39]:
#data preprocessing
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


#nltk.download('stopwords')



X = data["emails"]
Y = data["labels"]

def preprocess(emails):
  cleaned = []
  for i in emails:
    text = "".join(i).lower()
    text = re.sub(r"<.*?>", "", text)        # remove HTML
    text = re.sub(r"[^\w\s]", "", text)      # remove punctuation
    text = re.sub(r"\d+", "", text)          # remove numbers
    text = re.sub(r"\s+", " ", text).strip() # remove extra spaces
    cleaned.append(text)

  return cleaned
XP = preprocess(X)


# tokenizing ["hello word"] -> ["hello", "world"]
Tokenized = [i.split(" ") for i in XP]


# removing stop words
stop_words = set(stopwords.words("english"))
stops = [[i for i in tokens if i not in stop_words] for tokens in Tokenized]



#applying stemming to reduce the word into it's simple form
stemmer = PorterStemmer()

stemmed = [[stemmer.stem(i) for i in tokens] for tokens in stops]



#now apply text vectorization "convert the text into machine learning readable numbers"
# we apply counter vectorize:
vectorizer = CountVectorizer()
emails = [" ".join(i) for i in stemmed]
X_counts = vectorizer.fit_transform(emails)
X_counts

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 223016 stored elements and shape (4321, 18084)>

In [94]:
#split the data to training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_counts,Y, test_size = 0.20, random_state = 0)

# train a logistic regression:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(class_weight = "balanced",random_state= 9)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [96]:
#evaluate the model
# the test has approximately 93% accuracy rate which is great.

from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[242  23]
 [ 31 569]]


0.9375722543352601

In [108]:
#testing for fun

my = input("enter your review: ")
myclean = my.lower()
X_new = vectorizer.transform([myclean])
prediction = classifier.predict(X_new)
print("positive" if prediction[0] == 1 else "negative")

enter your review: hi
positive


try alternative method (Support vector Machine)

In [101]:

from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(X_train,y_train)
y_predSVM = svm.predict(X_test)
cmSVM = confusion_matrix(y_test,y_predSVM)
cmSVM
accuracy_score(y_test,y_predSVM)

0.9098265895953758

logistic regression got a higher accuracy (93.75%) compared to SVM (90.98%). It also had fewer false positives and false negatives which means it made fewer mistakes overall. logistic regression predicted both positive and negative reviews more correctly than SVM.Therefore,Logistic Regression is better.